In [1]:
import pandas as pd
import numpy as np
import math
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestRegressor

# Part 0: Load the data

In [2]:
radar_data = pd.read_csv('./homework/radar_parameters.csv')
radar_data.drop(columns='Unnamed: 0', inplace=True) # Unnecessary column

In [3]:
radar_data

,Zh (dBZ),Zdr (dB),Ldr (dB),Kdp (deg km-1),Ah (dBZ/km),Adr (dB/km),R (mm/hr)
0,23.144878,0.418637,-41.757733,0.005395,0.000290,0.000012,2.393520
1,22.737156,0.322850,-43.772069,0.005194,0.000360,0.000012,3.502699
2,26.869826,0.330948,-43.577399,0.013385,0.000903,0.000030,8.627561
3,28.540561,0.399480,-42.139731,0.018872,0.001036,0.000043,8.424447
4,30.500127,0.543758,-39.763087,0.027438,0.001157,0.000064,8.189291
...,...,...,...,...,...,...,...
18964,31.515997,0.579955,-39.244229,0.034048,0.001417,0.000080,10.648020
18965,29.993334,0.567935,-39.399188,0.024134,0.001032,0.000057,7.981875
18966,31.685913,0.655681,-38.375696,0.033971,0.001165,0.000081,6.822691
18967,32.980096,0.768586,-37.166218,0.043117,0.001285,0.000105,6.801169


# Part 1: Split the data

In [4]:
regressors = ['Zh (dBZ)', 'Zdr (dB)', 'Ldr (dB)', 'Kdp (deg km-1)', 'Ah (dBZ/km)', 'Adr (dB/km)', 'R (mm/hr)']
response = 'R (mm/hr)'
X = radar_data[regressors]
y = radar_data[response]

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)

# Part 2: Build and evaluate initial model

In [6]:
initial_model = LinearRegression()
initial_model.fit(X_train, y_train)
yhat = initial_model.predict(X_train)

In [7]:
SS_Residual = sum((y_train-yhat)**2)       
SS_Total = sum((y_train-np.mean(y_train))**2)     
r_squared = 1 - (float(SS_Residual))/SS_Total
adjusted_r_squared = 1 - (1-r_squared)*(len(y)-1)/(len(y)-X.shape[1]-1)
mse_train=np.mean((y_train-yhat)**2)
print(f'R2={r_squared}; Adjusted R2={adjusted_r_squared}; MSE={mse_train}; RMSE={math.sqrt(mse_train)}')

R2=1.0; Adjusted R2=1.0; MSE=7.305199936590789e-30; RMSE=2.7028133373562424e-15


In [8]:
SS_Residual

9.69984447580525e-26

### Residuals are very, very small

In [9]:
y_test_hat = initial_model.predict(X_test)
mse_test=np.mean((y_test-y_test_hat)**2)
print(f'MSE={mse_test}; RMSE={math.sqrt(mse_test)}')

MSE=6.823841541645288e-30; RMSE=2.6122483690578292e-15


In [10]:
print(y_train-yhat)

3144    -7.105427e-15
10696    8.881784e-16
6911    -4.440892e-15
16905    1.776357e-15
293      8.881784e-16
             ...     
13435    1.776357e-15
7763     0.000000e+00
15377    1.776357e-15
17730   -3.552714e-15
15725   -3.552714e-15
Name: R (mm/hr), Length: 13278, dtype: float64


### Residuals are consistent with the calculated RMSE

# Part 3: Do grid search over polynomial orders

In [11]:
polynomial_grid = GridSearchCV(make_pipeline(StandardScaler(), PolynomialFeatures(2), LinearRegression()), 
                               {'polynomialfeatures__degree': np.arange(8), 'linearregression__fit_intercept': [True]},
                               n_jobs=-1, scoring='neg_mean_squared_error')
polynomial_grid.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('standardscaler', StandardScaler()),
                                       ('polynomialfeatures',
                                        PolynomialFeatures()),
                                       ('linearregression',
                                        LinearRegression())]),
             n_jobs=-1,
             param_grid={'linearregression__fit_intercept': [True],
                         'polynomialfeatures__degree': array([0, 1, 2, 3, 4, 5, 6, 7])},
             scoring='neg_mean_squared_error')

In [12]:
print(polynomial_grid.best_params_)
best_linear_model = polynomial_grid.best_estimator_
print(best_linear_model)

{'linearregression__fit_intercept': True, 'polynomialfeatures__degree': 1}
Pipeline(steps=[('standardscaler', StandardScaler()),
                ('polynomialfeatures', PolynomialFeatures(degree=1)),
                ('linearregression', LinearRegression())])


In [13]:
yhat = best_linear_model.predict(X_train)
SS_Residual = sum((y_train-yhat)**2)       
SS_Total = sum((y_train-np.mean(y_train))**2)     
r_squared = 1 - (float(SS_Residual))/SS_Total
adjusted_r_squared = 1 - (1-r_squared)*(len(y)-1)/(len(y)-X.shape[1]-1)
mse_train=np.mean((y_train-yhat)**2)
print(f'R2={r_squared}; Adjusted R2={adjusted_r_squared}; MSE={mse_train}; RMSE={math.sqrt(mse_train)}')

R2=1.0; Adjusted R2=1.0; MSE=2.323517682827621e-28; RMSE=1.5243089197494126e-14


### MSE/RMSE is higher than the base model by an order of magnitude

# Part 4: Do grid search with random forest

In [14]:
rf_params= {'bootstrap': [True, False],  
   'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],  
   'max_features': [1, 'sqrt'],  
   'min_samples_leaf': [1, 2, 4],  
   'min_samples_split': [2, 5, 10],  
   'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}
rf_base = RandomForestRegressor(random_state=123)
rf_grid = RandomizedSearchCV(estimator=rf_base, param_distributions=rf_params, n_iter=100, cv=5, n_jobs=-1)
rf_grid.fit(X_train, y_train)

/home/brad/.local/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(random_state=123),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, None],
                                        'max_features': [1, 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]})

In [15]:
print(rf_grid.best_params_)
best_rf_model = rf_grid.best_estimator_
print(best_rf_model)

{'n_estimators': 1000, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 40, 'bootstrap': False}
RandomForestRegressor(bootstrap=False, max_depth=40, max_features='sqrt',
                      n_estimators=1000, random_state=123)


In [16]:
yhat = best_rf_model.predict(X_train)
SS_Residual = sum((y_train-yhat)**2)       
SS_Total = sum((y_train-np.mean(y_train))**2)     
r_squared = 1 - (float(SS_Residual))/SS_Total
adjusted_r_squared = 1 - (1-r_squared)*(len(y)-1)/(len(y)-X.shape[1]-1)
mse_train=np.mean((y_train-yhat)**2)
print(f'R2={r_squared}; Adjusted R2={adjusted_r_squared}; MSE={mse_train}; RMSE={math.sqrt(mse_train)}')

R2=0.9999999999999983; Adjusted R2=0.9999999999999983; MSE=1.3509521103839896e-13; RMSE=3.675530043931065e-07


### Performance of the random forest regressor is much worse.